In [1]:
import numpy as np
import pandas as pd
title = 'FuncLearning'

# das = pd.Series([0, 0, np.nan, 1, 1, 0, np.nan, 1, 0, 1], name='test')
das = pd.Series(['2G', '2G', '4G', '3G', '3G', '2G', np.nan, '3G', '5G', '3G'], name='test')
das

0     2G
1     2G
2     4G
3     3G
4     3G
5     2G
6    NaN
7     3G
8     5G
9     3G
Name: test, dtype: object

### 1. 计算各取值value_counts(注意不包括缺失值NaN), 根据预设门限a（大于，开区间）确定哪些取值参与Cat

In [2]:
a = 0
# a = 0.1
tmp = pd.value_counts(das)/das.shape[0]
print tmp
cat0 = list(tmp.index[tmp > 0])
cat1 = list(tmp.index[tmp > 0.1])
print 'cat of a=0:  ', cat0
print 'cat of a=0.1:', cat1

3G    0.4
2G    0.3
5G    0.1
4G    0.1
Name: test, dtype: float64
cat of a=0:   ['3G', '2G', '5G', '4G']
cat of a=0.1: ['3G', '2G']


### 2. 对Cat排序

In [3]:
cat = cat0 # attention:内存共用！
cat.sort()
# cat.sort(reverse=True)
cat

['2G', '3G', '4G', '5G']

### 3. 字段类型转category（如果不在Cat列表内，也将被标记为Nan）

In [4]:
print das.astype('category')
das.astype('category').cat.set_categories(cat)

0     2G
1     2G
2     4G
3     3G
4     3G
5     2G
6    NaN
7     3G
8     5G
9     3G
Name: test, dtype: category
Categories (4, object): [2G, 3G, 4G, 5G]


0     2G
1     2G
2     4G
3     3G
4     3G
5     2G
6    NaN
7     3G
8     5G
9     3G
dtype: category
Categories (4, object): [2G, 3G, 4G, 5G]

### 4. category字段重编码：从1开始，缺失值置0

In [5]:
df_test_cat = np.transpose([das.astype('category').cat.set_categories(cat).cat.rename_categories(1+np.arange(len(cat))).astype('float').fillna(0).values])
df_test_cat

array([[ 1.],
       [ 1.],
       [ 3.],
       [ 2.],
       [ 2.],
       [ 1.],
       [ 0.],
       [ 2.],
       [ 4.],
       [ 2.]])

### 执行OneHotEncoder
n_values中+1即缺失值？  
因为category中0表示缺失值，所以xbin的列切片从1开始更合理

In [6]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(n_values = len(cat)+1, sparse = False)
xbin = enc.fit_transform(df_test_cat)[:,1:]
xbin

array([[ 1.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.],
       [ 0.,  1.,  0.,  0.],
       [ 0.,  1.,  0.,  0.],
       [ 1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  1.],
       [ 0.,  1.,  0.,  0.]])

### 生成dataframe

In [7]:
dabin = pd.DataFrame(xbin, columns = ["{}_{}".format(das.name, x) for x in cat], index = das.index)
dabin

,test_2G,test_3G,test_4G,test_5G
0,1.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0
3,0.0,1.0,0.0,0.0
4,0.0,1.0,0.0,0.0
5,1.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0
7,0.0,1.0,0.0,0.0
8,0.0,0.0,0.0,1.0
9,0.0,1.0,0.0,0.0


### 判断不满足门限a（小于，闭区间）的其余列占比之和是否满足门限（大于，开区间）

In [8]:
print tmp
print tmp[tmp <= a].sum()
# if(tmp[tmp <= a].sum() > a):
#     dabin = pd.concat([dabin, pd.DataFrame({"{}_Others".format(das.name):das.notnull()-dabin.sum(axis = 1)})], axis = 1)

3G    0.4
2G    0.3
5G    0.1
4G    0.1
Name: test, dtype: float64
0


### 如果满足，进一步分析
分析das.notnull()的取值
* if 原始数据非Nan, das.notnull() == true
* if 原始数据Nan  , das.notnull() == false

分析dabin.sum(axis = 1)的取值
* if 原始数据取值属于Cat, 则dabin.sum(axis = 1) == 1, 即分箱后一个元素置1,其余置0
* if 原始数据取值不属于Cat 则dabin.sum(axis = 1) == 0, 即分箱后元素全部置0

In [9]:
print das,'\n'
print das.notnull(), '\n'
print dabin.sum(axis=1)

pd.DataFrame({"{}_Others".format(das.name):das.notnull()-dabin.sum(axis = 1)})

0     2G
1     2G
2     4G
3     3G
4     3G
5     2G
6    NaN
7     3G
8     5G
9     3G
Name: test, dtype: object 

0     True
1     True
2     True
3     True
4     True
5     True
6    False
7     True
8     True
9     True
Name: test, dtype: bool 

0    1.0
1    1.0
2    1.0
3    1.0
4    1.0
5    1.0
6    0.0
7    1.0
8    1.0
9    1.0
dtype: float64


,test_Others
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
5,0.0
6,0.0
7,0.0
8,0.0
9,0.0


In [10]:
if(tmp[tmp <= a].sum() > a):
    dabin = pd.concat([dabin, pd.DataFrame({"{}_Others".format(das.name):das.notnull()-dabin.sum(axis = 1)})], axis = 1)
dabin

,test_2G,test_3G,test_4G,test_5G
0,1.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0
3,0.0,1.0,0.0,0.0
4,0.0,1.0,0.0,0.0
5,1.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0
7,0.0,1.0,0.0,0.0
8,0.0,0.0,0.0,1.0
9,0.0,1.0,0.0,0.0


In [11]:
print dabin.shape
if(dabin.shape[1] == 2):
    dabin = pd.DataFrame({das.name: xbin[:,0]}, index = das.index)
dabin.head()

(10, 4)


,test_2G,test_3G,test_4G,test_5G
0,1.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0
3,0.0,1.0,0.0,0.0
4,0.0,1.0,0.0,0.0
